In [ ]:
from __future__ import print_function

import keras
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Add
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import BatchNormalization as BN
from keras.layers import GaussianNoise as GN
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import LearningRateScheduler as LRS
from keras.preprocessing.image import ImageDataGenerator


batch_size = 128
num_classes = 10
epochs = 50


#### LOAD AND TRANSFORM
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print(x_train.shape)
print(x_test.shape)

y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)



## DEFINE A DATA AUGMENTATION GENERATOR

datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True)


## DEF A BLOCK RES + CONV + BN + GN + MAXPOOL
def resnet_layer(inputs, filters):
    x = inputs
    x = Conv2D(filters, (3, 3), padding='same')(x)
    x = BN()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, (3, 3), padding='same')(x)
    x = BN()(x)
    previous = Conv2D(filters, (1,1), padding='same')(inputs)
    x = Add()([previous,x])
    x = Activation('relu')(x)

    return x

  
## DEF RESNET TOPOLOGY  
inputs = Input(shape=x_train.shape[1:])
x = resnet_layer(inputs, 64)
x = resnet_layer(x, 64)
x = resnet_layer(x, 128)
x = resnet_layer(x, 128)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = resnet_layer(x, 256)
x = resnet_layer(x, 256)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = resnet_layer(x, 512)
x = resnet_layer(x, 512)
x = AveragePooling2D(pool_size=(8, 8))(x)
x = Flatten()(x)
x = Dense(num_classes)(x)
x = Activation('softmax')(x)

model = Model(inputs=inputs, outputs=x)
model.summary()


## OPTIM AND COMPILE
opt = Adam(learning_rate=0.001)


model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# DEFINE A LEARNING RATE SCHEDULER
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=0.00001)

## TRAINING with DA and LRA
history=model.fit(datagen.flow(x_train, y_train,batch_size=batch_size),
                            steps_per_epoch=len(x_train) / batch_size, 
                            epochs=epochs,
                            validation_data=(x_test, y_test),
                            callbacks=[reduce_lr],
                            verbose=1)


## TEST
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

170508288/170498071 [==============================] - 11s 0us/step
(50000, 32, 32, 3)
(10000, 32, 32, 3)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 64)   1792        ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 64)  256         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                       